In [1]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pytest
from matplotlib.gridspec import GridSpec
from scipy.stats import randint, uniform
from sklearn.model_selection import ShuffleSplit
from keras_tuner import HyperParameters as hp

import pyMAISE as mai

settings = {
    "verbosity": 1,
    "random_state": 42,
    "test_size": 0.3,
    "num_configs_saved": 5,
    "regression": True,
    "cuda_visible_devices": "-1",
}


global_settings = mai.settings.init(settings_changes=settings)

preprocessor = mai.load_xs()

data = preprocessor.min_max_scale()

Using TensorFlow backend


2023-09-27 11:55:40.979733: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-27 11:55:41.002805: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-27 11:55:41.373510: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Num GPUs Available:  0
Reading data from /home/myerspat/dev/pyMAISE/pyMAISE/data/xs.csv
MinMax scaling data


2023-09-27 11:55:41.865169: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


## pyMAISE Initialization

First we initialize pyMAISE with the following 4 parameters:

- `verbosity`: 0 $\rightarrow$ pyMAISE prints no outputs,
- `random_state`: None $\rightarrow$ No random seed is set,
- `test_size`: 0.3 $\rightarrow$ 30% of the data is used for testing,
- `num_configs_saved`: 5 $\rightarrow$ The top 5 hyper-parameter configurations are saved for each model.

With pyMAISE initialized we can load the preprocessor for this data set using `load_fp()`.

In [2]:
structural_hyperparameters = {
    "dense_input": {
        "units": mai.Int(min_value=25, max_value=300, step=10),
        "input_dim": preprocessor.inputs.shape[1],
        "activation": "relu",
        "kernel_initializer": "normal",
    },
    "dense_hidden": {
        "units": mai.Int(min_value=25, max_value=300, step=10),
        "activation": "relu",
        "kernel_initializer": "normal",
        "num_layers": mai.Int(min_value=1, max_value=3),
    },
    "dense_output": {
        "units": preprocessor.outputs.shape[1],
        "input_dim": None,
        "activation": "linear",
        "kernel_initializer": "normal",
    },
}

adam = {
    "learning_rate": 1e-3,
}


model_settings = {
    "models": ["linear", "fnn"],
    "fnn": {
        "structural_params": structural_hyperparameters,
        "optimizer": "adam",
        "adam": adam,
        "compile_params": {
            "loss": "mean_absolute_error",
            "metrics": ["mean_absolute_error"],
        },
        "fitting_params": {
            "batch_size": 8,
            "epochs": 50,
            "validation_split": 0.15,
        },
    },
}

tuning = mai.Tuning(data=data, model_settings=model_settings)
bayesian_search = tuning.nn_bayesian_search(
    models = ["fnn"],
    objective = 'mean_absolute_error',
    max_trials = 10,
)

Trial 10 Complete [00h 00m 05s]
mean_absolute_error: 0.005094194784760475

Best mean_absolute_error So Far: 0.0032120889518409967
Total elapsed time: 00h 00m 43s
-- fnn
Search space summary
Default search space size: 5
dense_input0_units (Int)
{'default': None, 'conditions': [], 'min_value': 25, 'max_value': 300, 'step': 10, 'sampling': 'linear'}
dense_hidden_num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 3, 'step': 1, 'sampling': 'linear'}
dense_hidden0_units (Int)
{'default': None, 'conditions': [], 'min_value': 25, 'max_value': 300, 'step': 10, 'sampling': 'linear'}
dense_hidden1_units (Int)
{'default': None, 'conditions': [], 'min_value': 25, 'max_value': 300, 'step': 10, 'sampling': 'linear'}
dense_hidden2_units (Int)
{'default': None, 'conditions': [], 'min_value': 25, 'max_value': 300, 'step': 10, 'sampling': 'linear'}
None
Results summary
Results in ./best_hp
Showing 10 best trials
Objective(name="mean_absolute_error", direction="min")

Trial 

In [ ]:
new_model_settings = {
    "fnn": {
        "fitting_params":{
            "epochs": 200,
        }
    }
}
postprocessor = mai.PostProcessor(
    data=data,
    models_list=[bayesian_search],
    new_model_settings=new_model_settings,
)

Epoch 1/200
75/75 [==============================] - 0s 2ms/step - loss: 0.1672 - mean_absolute_error: 0.1672 - val_loss: 0.1033 - val_mean_absolute_error: 0.1033
Epoch 2/200
75/75 [==============================] - 0s 990us/step - loss: 0.0573 - mean_absolute_error: 0.0573 - val_loss: 0.0347 - val_mean_absolute_error: 0.0347
Epoch 3/200
75/75 [==============================] - 0s 904us/step - loss: 0.0233 - mean_absolute_error: 0.0233 - val_loss: 0.0134 - val_mean_absolute_error: 0.0134
Epoch 4/200
75/75 [==============================] - 0s 894us/step - loss: 0.0182 - mean_absolute_error: 0.0182 - val_loss: 0.0118 - val_mean_absolute_error: 0.0118
Epoch 5/200
75/75 [==============================] - 0s 908us/step - loss: 0.0110 - mean_absolute_error: 0.0110 - val_loss: 0.0140 - val_mean_absolute_error: 0.0140
Epoch 6/200
75/75 [==============================] - 0s 981us/step - loss: 0.0132 - mean_absolute_error: 0.0132 - val_loss: 0.0224 - val_mean_absolute_error: 0.0224
Epoch 7/200


75/75 [==============================] - 0s 967us/step - loss: 0.0036 - mean_absolute_error: 0.0036 - val_loss: 0.0042 - val_mean_absolute_error: 0.0042
Epoch 51/200
75/75 [==============================] - 0s 984us/step - loss: 0.0071 - mean_absolute_error: 0.0071 - val_loss: 0.0041 - val_mean_absolute_error: 0.0041
Epoch 52/200
75/75 [==============================] - 0s 888us/step - loss: 0.0084 - mean_absolute_error: 0.0084 - val_loss: 0.0050 - val_mean_absolute_error: 0.0050
Epoch 53/200
75/75 [==============================] - 0s 888us/step - loss: 0.0050 - mean_absolute_error: 0.0050 - val_loss: 0.0033 - val_mean_absolute_error: 0.0033
Epoch 54/200
75/75 [==============================] - 0s 887us/step - loss: 0.0062 - mean_absolute_error: 0.0062 - val_loss: 0.0114 - val_mean_absolute_error: 0.0114
Epoch 55/200
75/75 [==============================] - 0s 907us/step - loss: 0.0063 - mean_absolute_error: 0.0063 - val_loss: 0.0038 - val_mean_absolute_error: 0.0038
Epoch 56/200
75/7

75/75 [==============================] - 0s 898us/step - loss: 0.0037 - mean_absolute_error: 0.0037 - val_loss: 0.0075 - val_mean_absolute_error: 0.0075
Epoch 100/200
75/75 [==============================] - 0s 994us/step - loss: 0.0064 - mean_absolute_error: 0.0064 - val_loss: 0.0024 - val_mean_absolute_error: 0.0024
Epoch 101/200
75/75 [==============================] - 0s 928us/step - loss: 0.0039 - mean_absolute_error: 0.0039 - val_loss: 0.0039 - val_mean_absolute_error: 0.0039
Epoch 102/200
75/75 [==============================] - 0s 949us/step - loss: 0.0036 - mean_absolute_error: 0.0036 - val_loss: 0.0073 - val_mean_absolute_error: 0.0073
Epoch 103/200
75/75 [==============================] - 0s 890us/step - loss: 0.0037 - mean_absolute_error: 0.0037 - val_loss: 0.0040 - val_mean_absolute_error: 0.0040
Epoch 104/200
75/75 [==============================] - 0s 893us/step - loss: 0.0057 - mean_absolute_error: 0.0057 - val_loss: 0.0037 - val_mean_absolute_error: 0.0037
Epoch 105/20

75/75 [==============================] - 0s 966us/step - loss: 0.0042 - mean_absolute_error: 0.0042 - val_loss: 0.0020 - val_mean_absolute_error: 0.0020
Epoch 149/200
75/75 [==============================] - 0s 891us/step - loss: 0.0042 - mean_absolute_error: 0.0042 - val_loss: 0.0073 - val_mean_absolute_error: 0.0073
Epoch 150/200
75/75 [==============================] - 0s 884us/step - loss: 0.0055 - mean_absolute_error: 0.0055 - val_loss: 0.0036 - val_mean_absolute_error: 0.0036
Epoch 151/200
75/75 [==============================] - 0s 906us/step - loss: 0.0033 - mean_absolute_error: 0.0033 - val_loss: 0.0016 - val_mean_absolute_error: 0.0016
Epoch 152/200
75/75 [==============================] - 0s 907us/step - loss: 0.0042 - mean_absolute_error: 0.0042 - val_loss: 0.0013 - val_mean_absolute_error: 0.0013
Epoch 153/200
75/75 [==============================] - 0s 872us/step - loss: 0.0043 - mean_absolute_error: 0.0043 - val_loss: 0.0015 - val_mean_absolute_error: 0.0015
Epoch 154/20

75/75 [==============================] - 0s 912us/step - loss: 0.0033 - mean_absolute_error: 0.0033 - val_loss: 0.0014 - val_mean_absolute_error: 0.0014
Epoch 198/200
75/75 [==============================] - 0s 888us/step - loss: 0.0033 - mean_absolute_error: 0.0033 - val_loss: 0.0107 - val_mean_absolute_error: 0.0107
Epoch 199/200
75/75 [==============================] - 0s 886us/step - loss: 0.0030 - mean_absolute_error: 0.0030 - val_loss: 0.0015 - val_mean_absolute_error: 0.0015
Epoch 200/200
10/10 [==============================] - 0s 757us/step
Epoch 1/200
75/75 [==============================] - 0s 2ms/step - loss: 0.2764 - mean_absolute_error: 0.2764 - val_loss: 0.1727 - val_mean_absolute_error: 0.1727
Epoch 2/200
75/75 [==============================] - 0s 1ms/step - loss: 0.1353 - mean_absolute_error: 0.1353 - val_loss: 0.1333 - val_mean_absolute_error: 0.1333
Epoch 3/200
75/75 [==============================] - 0s 925us/step - loss: 0.0940 - mean_absolute_error: 0.0940 - val_

...

75/75 [==============================] - 0s 894us/step - loss: 0.0056 - mean_absolute_error: 0.0056 - val_loss: 0.0045 - val_mean_absolute_error: 0.0045
Epoch 88/200
75/75 [==============================] - 0s 880us/step - loss: 0.0066 - mean_absolute_error: 0.0066 - val_loss: 0.0029 - val_mean_absolute_error: 0.0029
Epoch 89/200
75/75 [==============================] - 0s 888us/step - loss: 0.0067 - mean_absolute_error: 0.0067 - val_loss: 0.0063 - val_mean_absolute_error: 0.0063
Epoch 90/200
75/75 [==============================] - 0s 1ms/step - loss: 0.0048 - mean_absolute_error: 0.0048 - val_loss: 0.0013 - val_mean_absolute_error: 0.0013
Epoch 91/200
75/75 [==============================] - 0s 913us/step - loss: 0.0065 - mean_absolute_error: 0.0065 - val_loss: 0.0058 - val_mean_absolute_error: 0.0058
Epoch 92/200
75/75 [==============================] - 0s 878us/step - loss: 0.0045 - mean_absolute_error: 0.0045 - val_loss: 0.0056 - val_mean_absolute_error: 0.0056
Epoch 93/200
75/75 

75/75 [==============================] - 0s 901us/step - loss: 0.0036 - mean_absolute_error: 0.0036 - val_loss: 0.0080 - val_mean_absolute_error: 0.0080
Epoch 137/200
75/75 [==============================] - 0s 888us/step - loss: 0.0045 - mean_absolute_error: 0.0045 - val_loss: 0.0022 - val_mean_absolute_error: 0.0022
Epoch 138/200
75/75 [==============================] - 0s 890us/step - loss: 0.0026 - mean_absolute_error: 0.0026 - val_loss: 0.0031 - val_mean_absolute_error: 0.0031
Epoch 139/200
75/75 [==============================] - 0s 877us/step - loss: 0.0044 - mean_absolute_error: 0.0044 - val_loss: 0.0085 - val_mean_absolute_error: 0.0085
Epoch 140/200
75/75 [==============================] - 0s 899us/step - loss: 0.0047 - mean_absolute_error: 0.0047 - val_loss: 0.0033 - val_mean_absolute_error: 0.0033
Epoch 141/200
75/75 [==============================] - 0s 878us/step - loss: 0.0067 - mean_absolute_error: 0.0067 - val_loss: 0.0026 - val_mean_absolute_error: 0.0026
Epoch 142/20

75/75 [==============================] - 0s 886us/step - loss: 0.0047 - mean_absolute_error: 0.0047 - val_loss: 0.0049 - val_mean_absolute_error: 0.0049
Epoch 186/200
75/75 [==============================] - 0s 903us/step - loss: 0.0063 - mean_absolute_error: 0.0063 - val_loss: 0.0048 - val_mean_absolute_error: 0.0048
Epoch 187/200
75/75 [==============================] - 0s 875us/step - loss: 0.0042 - mean_absolute_error: 0.0042 - val_loss: 0.0030 - val_mean_absolute_error: 0.0030
Epoch 188/200
75/75 [==============================] - 0s 859us/step - loss: 0.0030 - mean_absolute_error: 0.0030 - val_loss: 0.0042 - val_mean_absolute_error: 0.0042
Epoch 189/200
75/75 [==============================] - 0s 875us/step - loss: 0.0045 - mean_absolute_error: 0.0045 - val_loss: 0.0024 - val_mean_absolute_error: 0.0024
Epoch 190/200
75/75 [==============================] - 0s 923us/step - loss: 0.0046 - mean_absolute_error: 0.0046 - val_loss: 0.0038 - val_mean_absolute_error: 0.0038
Epoch 191/20

Epoch 34/200
75/75 [==============================] - 0s 950us/step - loss: 0.0049 - mean_absolute_error: 0.0049 - val_loss: 0.0016 - val_mean_absolute_error: 0.0016
Epoch 35/200
75/75 [==============================] - 0s 1000us/step - loss: 0.0060 - mean_absolute_error: 0.0060 - val_loss: 0.0133 - val_mean_absolute_error: 0.0133
Epoch 36/200
75/75 [==============================] - 0s 944us/step - loss: 0.0059 - mean_absolute_error: 0.0059 - val_loss: 0.0042 - val_mean_absolute_error: 0.0042
Epoch 37/200
75/75 [==============================] - 0s 954us/step - loss: 0.0053 - mean_absolute_error: 0.0053 - val_loss: 0.0106 - val_mean_absolute_error: 0.0106
Epoch 38/200
75/75 [==============================] - 0s 911us/step - loss: 0.0078 - mean_absolute_error: 0.0078 - val_loss: 0.0084 - val_mean_absolute_error: 0.0084
Epoch 39/200
75/75 [==============================] - 0s 879us/step - loss: 0.0033 - mean_absolute_error: 0.0033 - val_loss: 0.0048 - val_mean_absolute_error: 0.0048
Epo

75/75 [==============================] - 0s 1ms/step - loss: 0.0043 - mean_absolute_error: 0.0043 - val_loss: 0.0090 - val_mean_absolute_error: 0.0090
Epoch 84/200
75/75 [==============================] - 0s 890us/step - loss: 0.0066 - mean_absolute_error: 0.0066 - val_loss: 0.0123 - val_mean_absolute_error: 0.0123
Epoch 85/200
75/75 [==============================] - 0s 889us/step - loss: 0.0057 - mean_absolute_error: 0.0057 - val_loss: 0.0072 - val_mean_absolute_error: 0.0072
Epoch 86/200
75/75 [==============================] - 0s 886us/step - loss: 0.0039 - mean_absolute_error: 0.0039 - val_loss: 0.0022 - val_mean_absolute_error: 0.0022
Epoch 87/200
75/75 [==============================] - 0s 864us/step - loss: 0.0039 - mean_absolute_error: 0.0039 - val_loss: 0.0042 - val_mean_absolute_error: 0.0042
Epoch 88/200
75/75 [==============================] - 0s 865us/step - loss: 0.0062 - mean_absolute_error: 0.0062 - val_loss: 0.0052 - val_mean_absolute_error: 0.0052
Epoch 89/200
75/75 

In [ ]:
postprocessor.metrics()

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
ax = postprocessor.nn_learning_plot()